In [ ]:
import pandas as pd

csv_path = 'csv/SPY_3d_quotes.csv'
df_price = pd.read_csv(csv_path)

df_price['ask_price'] = df_price['ap']
df_price['bid_price'] = df_price['bp']

df_price['ask_size'] = df_price['as']
df_price['bid_size'] = df_price['bs']

df_price['mid_price'] = (df_price['ask_price'] + df_price['bid_price']) / 2

df_price['spread_ask_bid'] = ((df_price['ask_price'] - df_price['bid_price']) / ((df_price['ask_price'] + df_price['ask_price']) / 2)) * 100

# df_price = df_price[900000:1800000]

df_price['time'] = pd.to_datetime(df_price['t'], unit='ns', utc=True)

df_price['time'] = df_price['time'].dt.round('ms')
df_price['time'] = df_price['time'].astype('int64') // 10**6

columns_to_drop = ['as', 'bs', 'ap', 'bp', 't', 'c', 'z']
df_price.drop(columns=columns_to_drop, inplace=True)

df_price

,ax,bx,ask_price,bid_price,ask_size,bid_size,mid_price,spread_ask_bid,time
1,T,T,541.62,541.46,1,1,541.540,0.029541,1744617600001
2,T,K,541.62,536.08,1,50,538.850,1.022857,1744617600014
3,T,K,541.62,537.39,1,1,539.505,0.780990,1744617600015
4,T,K,541.62,536.08,1,50,538.850,1.022857,1744617600015
5,K,K,540.00,536.08,1,50,538.040,0.725926,1744617600015
...,...,...,...,...,...,...,...,...,...
12126686,P,P,526.00,525.93,4,22,525.965,0.013308,1744847998597
12126687,P,P,526.00,525.94,4,18,525.970,0.011407,1744847998597
12126688,P,Z,526.00,525.93,4,21,525.965,0.013308,1744847998599
12126689,P,P,526.00,525.94,4,1,525.970,0.011407,1744847998788


In [63]:
import pandas as pd

sorted_spreads = df_price['spread_ask_bid'].sort_values(ascending=False)
top_30_percent = sorted_spreads.head(int(len(df_price) * 0.1))
mean_top_spreads = top_30_percent.mean()
print(mean_top_spreads)

0.019883989992011693


In [ ]:
import pandas as pd

df = df_price
q0 = df['spread_ask_bid'].quantile(0.05)
q1 = df['spread_ask_bid'].quantile(0.10)
q2 = df['spread_ask_bid'].quantile(0.20)
q3 = df['spread_ask_bid'].quantile(0.50)
q4 = df['spread_ask_bid'].quantile(0.75)



quartile_0 = df[df['spread_ask_bid'] <= q0]['spread_ask_bid']
quartile_1 = df[(df['spread_ask_bid'] > q0) & (df['spread_ask_bid'] <= q1)]['spread_ask_bid']
quartile_2 = df[(df['spread_ask_bid'] > q1) & (df['spread_ask_bid'] <= q2)]['spread_ask_bid']
quartile_3 = df[(df['spread_ask_bid'] > q2) & (df['spread_ask_bid'] <= q3)]['spread_ask_bid']
quartile_4 = df[df['spread_ask_bid'] > q3]['spread_ask_bid']


mean_q0 = quartile_0.mean()
mean_q1 = quartile_1.mean()
mean_q2 = quartile_2.mean()
mean_q3 = quartile_3.mean()
mean_q4 = quartile_4.mean()


print("Moyenne Q0 (0-5%) :", mean_q0)
print("Moyenne Q1 (5-10%) :", mean_q1)
print("Moyenne Q2 (10-20%) :", mean_q2)
print("Moyenne Q3 (20-50%) :", mean_q3)
print("Moyenne Q4 (50-100%) :", mean_q4)

Moyenne Q0 (0-5%) : 0.0016890555167241592
Moyenne Q1 (5-10%) : 0.0032629177561451753
Moyenne Q2 (10-20%) : 0.003739340210110495
Moyenne Q3 (20-50%) : 0.005532939002429283
Moyenne Q4 (50-100%) : 0.01055433664851105


In [65]:
# import pandas as pd
# import plotly.graph_objs as go

# percentage_threshold = 0.002  # 0.005 MSTR, 

# data_time = 10  

# df = df_price
# df = df.sort_values('time').reset_index(drop=True)

# # Calcul des variations en pourcentage
# df['ask_pct_change'] = df['ask_price'].pct_change()
# df['bid_pct_change'] = df['bid_price'].pct_change()

# # Calcul de la différence de temps
# df['time_diff_ms'] = df['time'].diff()

# # Masque pour détecter les pics
# mask = (
#     ((df['ask_pct_change'].abs() > percentage_threshold) | (df['bid_pct_change'].abs() > percentage_threshold)) &
#     (df['time_diff_ms'] <= data_time)
# )

# # Filtrer les pics
# df_peaks = df[mask]

# # Inclure les lignes correspondantes pour le bid et l'ask au même timestamp
# df_spread_peaks = df[df['time'].isin(df_peaks['time'])].copy()

# # Convertir les timestamps en datetime pour l'axe x
# df['datetime'] = pd.to_datetime(df['time'], unit='ms')
# df_spread_peaks['datetime'] = pd.to_datetime(df_spread_peaks['time'], unit='ms')

# # Traces
# fig = go.Figure()

# # Tracer les prix ask
# fig.add_trace(go.Scatter(
#     x=df['datetime'],
#     y=df['ask_price'],
#     mode='lines',
#     line_shape='hv',
#     name='ask_price',
#     line=dict(color='blue', width=1)
# ))

# # Tracer les prix bid
# fig.add_trace(go.Scatter(
#     x=df['datetime'],
#     y=df['bid_price'],
#     mode='lines',
#     line_shape='hv',
#     name='bid_price',
#     line=dict(color='orange', width=1)
# ))

# # Marqueurs pour les pics (ask et bid au même timestamp)
# fig.add_trace(go.Scatter(
#     x=df_spread_peaks['datetime'],
#     y=df_spread_peaks['ask_price'],
#     mode='markers',
#     name='Ask Peaks',
#     marker=dict(color='red', size=6),
#     text=(df_spread_peaks['ask_pct_change'] * 100).round(3).astype(str) + '%',
#     hoverinfo='text+x+y'
# ))

# fig.add_trace(go.Scatter(
#     x=df_spread_peaks['datetime'],
#     y=df_spread_peaks['bid_price'],
#     mode='markers',
#     name='Bid Peaks',
#     marker=dict(color='green', size=6),
#     text=(df_spread_peaks['bid_pct_change'] * 100).round(3).astype(str) + '%',
#     hoverinfo='text+x+y'
# ))

# # Layout
# fig.update_layout(
#     title="Pics de prix avec spread",
#     xaxis_title="Temps",
#     yaxis_title="Prix",
#     hovermode='x unified',
#     template='plotly_white',
#     height=600
# )

# fig.show()